In [1]:
import secretflow as sf

# In case you have a running secretflow runtime already.
sf.shutdown()
# 基于这四个party，我们将会建立3个设备。
# 一个基于 alice 的PYU设备
# 一个基于 dave 的PYU设备
# 一个基于 alice , bob 和 carol 的SPU设备
sf.init(['alice', 'bob', 'carol', 'dave'], address='local')

2023-06-01 20:13:32,514	INFO worker.py:1538 -- Started a local Ray instance.


In [2]:
# sf.utils.testing.cluster_def 是一个helper通过寻找未占用的端口来创建一个设置。
aby3_config = sf.utils.testing.cluster_def(parties=['alice', 'bob', 'carol'])

aby3_config

{'nodes': [{'party': 'alice', 'address': '127.0.0.1:51451'},
  {'party': 'bob', 'address': '127.0.0.1:38011'},
  {'party': 'carol', 'address': '127.0.0.1:42141'}],
 'runtime_config': {'protocol': 3, 'field': 3}}

In [3]:
# 随后我们用 aby3_config 来创建一个SPU设备并检查其 cluster_def 。
spu_device = sf.SPU(aby3_config)

spu_device.cluster_def

{'nodes': [{'party': 'alice', 'address': '127.0.0.1:51451'},
  {'party': 'bob', 'address': '127.0.0.1:38011'},
  {'party': 'carol', 'address': '127.0.0.1:42141'}],
 'runtime_config': {'protocol': 3, 'field': 3}}

In [4]:
# 们创建两个PYU设备。
alice, dave = sf.PYU('alice'), sf.PYU('dave')

In [5]:
# 首先，我们用一个PYU设备创建一个PYU object。
def debit_amount():
    return 10


debit_amount_pyu = alice(debit_amount)()
debit_amount_pyu

In [6]:
# 然后我们将debit_amount_pyu从PYU传到SPU，我们将会得到一个SPU object作为结果。
debit_amount_spu = debit_amount_pyu.to(spu_device)

debit_amount_spu

(_run pid=24900) INFO:jax._src.xla_bridge:Unable to initialize backend 'cuda': module 'jaxlib.xla_extension' has no attribute 'GpuAllocatorConfig'
(_run pid=24900) INFO:jax._src.xla_bridge:Unable to initialize backend 'rocm': module 'jaxlib.xla_extension' has no attribute 'GpuAllocatorConfig'
(_run pid=24900) INFO:jax._src.xla_bridge:Unable to initialize backend 'tpu': INVALID_ARGUMENT: TpuPlatform is not available.
(_run pid=24900) INFO:jax._src.xla_bridge:Unable to initialize backend 'plugin': xla_extension has no attributes named get_plugin_device_client. Compile TensorFlow with //tensorflow/compiler/xla/python:enable_plugin_device set to true (defaults to false) to enable this.
(_run pid=24900) WARNING:jax._src.xla_bridge:No GPU/TPU found, falling back to CPU. (Set TF_CPP_MIN_LOG_LEVEL=0 and rerun for more info.)


In [16]:
# print(debit_amount_pyu.meta)
# pyu没有meta和sharenames好像，上面那个玩意会报错
print(dir(debit_amount_pyu))
print(debit_amount_pyu.data)


['__abstractmethods__', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__slots__', '__str__', '__subclasshook__', '__weakref__', '_abc_impl', 'data', 'device', 'device_type', 'to']
ObjectRef(32d950ec0ccf9d2affffffffffffffffffffffff0100000001000000)


(raylet) [2023-06-01 20:30:32,441 E 24794 24794] (raylet) node_manager.cc:3097: 1 Workers (tasks / actors) killed due to memory pressure (OOM), 0 Workers crashed due to other reasons at node (ID: 019f9c5ff5f0db414ccc6a7f58aeed1f0edf9097082fb2918e01d5b9, IP: 10.136.75.128) over the last time period. To see more information about the Workers killed on this node, use `ray logs raylet.out -ip 10.136.75.128`
(raylet) 
(raylet) Refer to the documentation on how to address the out of memory issue: https://docs.ray.io/en/latest/ray-core/scheduling/ray-oom-prevention.html. Consider provisioning more memory on this node or reducing task parallelism by requesting more CPUs per task. To adjust the kill threshold, set the environment variable `RAY_memory_usage_threshold` when starting Ray. To disable worker killing, set the environment variable `RAY_memory_monitor_refresh_ms` to zero.
(raylet) [2023-06-01 20:52:32,475 E 24794 24794] (raylet) node_manager.cc:3097: 1 Workers (tasks / actors) killed d

In [7]:
# 它是一个在alice一边的Ray ObjectRef
debit_amount_spu.meta

ObjectRef(e0dc174c83599034ffffffffffffffffffffffff0100000001000000)

In [8]:
# 会得到一个ObjectRef列表。因为它在alice这一侧，我们无法在host检查它的值。
# 不太懂什么叫无法在host检查他的值
debit_amount_spu.shares_name

[ObjectRef(f4402ec78d3a26076093ec3f85f7046ff7565a3d0100000001000000),
 ObjectRef(f91b78d7db9a65937bf1dc25d2421469a0f9c22b0100000001000000),
 ObjectRef(82891771158d68c152418d0ea3ad27761416c8f80100000001000000)]

In [9]:
sf.reveal(debit_amount_spu)

array(10, dtype=int32)

(raylet) [2023-06-01 20:20:32,426 E 24794 24794] (raylet) node_manager.cc:3097: 2 Workers (tasks / actors) killed due to memory pressure (OOM), 0 Workers crashed due to other reasons at node (ID: 019f9c5ff5f0db414ccc6a7f58aeed1f0edf9097082fb2918e01d5b9, IP: 10.136.75.128) over the last time period. To see more information about the Workers killed on this node, use `ray logs raylet.out -ip 10.136.75.128`
(raylet) 
(raylet) Refer to the documentation on how to address the out of memory issue: https://docs.ray.io/en/latest/ray-core/scheduling/ray-oom-prevention.html. Consider provisioning more memory on this node or reducing task parallelism by requesting more CPUs per task. To adjust the kill threshold, set the environment variable `RAY_memory_usage_threshold` when starting Ray. To disable worker killing, set the environment variable `RAY_memory_monitor_refresh_ms` to zero.
